# 100 TIMES AI WORLD BUILDING - Local Version v2.0

**完全ローカル実行版 - Ollama + gpt-oss:20b**

このノートブックは、AIを活用した世界観構築ワークフローを完全にローカル環境で実行します。

## 特徴

- ✅ 完全なプライバシー保護（データが外部に送信されない）
- ✅ ゼロコスト運用（API利用料金なし）
- ✅ オフライン動作（初回モデルダウンロード後）
- ✅ カスタマイズ性（モデルパラメータの細かい調整）

## 前提条件

1. Ollamaがインストール済み（`ollama serve`で起動）
2. gpt-oss:20bモデルがダウンロード済み（自動ダウンロードも可能）
3. 必要なPythonパッケージがインストール済み

## リンク

- [GitHub Repository](https://github.com/masa-jp-art/100-times-ai-world-building)
- [設計仕様書](DESIGN_SPEC_LOCAL.md)
- [README](README_LOCAL.md)

## Cell 1: セットアップと初期化

必要なライブラリのインポートと初期化を行います。

In [ ]:
# ライブラリのインポート
import sys
from pathlib import Path

# プロジェクトルートをパスに追加
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# コアモジュールのインポート
from src import (
    OllamaClient,
    CheckpointManager,
    Pipeline,
    load_config,
    setup_logging,
    rich_print,
)

# ロギングのセットアップ
setup_logging(
    log_level="INFO",
    log_file="./logs/local_v2.log",
    console=True
)

print("✓ ライブラリのインポート完了")

## Cell 2: パイプラインの初期化

パイプラインを初期化し、前提条件をチェックします。

In [ ]:
# パイプラインの初期化
pipeline = Pipeline(
    config_path="config/ollama_config.yaml",
    prompts_dir="config/prompts"
)

# 前提条件のチェック
if pipeline.check_prerequisites():
    print("\n✓ すべての前提条件が満たされています")
    print("パイプラインの実行準備が完了しました")
else:
    print("\n✗ 前提条件が満たされていません")
    print("エラーメッセージを確認してください")

## Cell 3: ユーザーコンテクストの入力

物語の元となるコンテクストを入力します。

In [ ]:
# ユーザーコンテクストの入力
# 以下のテンプレートを編集して、あなた独自のコンテクストを入力してください

user_context = """
context:
  theme: "未来都市での人間と人工知能の共存"
  mood: "希望と不安が交錯する"
  setting: "2080年代の東京"
  key_elements:
    - "完全自動化された社会"
    - "失われつつある人間性"
    - "新しい形のコミュニケーション"
  protagonist_idea: "AIと対話できる特殊能力を持つ若者"
"""

print("ユーザーコンテクスト:")
print(user_context)
print("\n✓ コンテクストが設定されました")

## Cell 4: Phase 0 - コンテクスト抽出（オプション）

対話形式でコンテクストを抽出する場合は、このセルを実行します。
上のセルで直接入力した場合は、このセルはスキップできます。

In [ ]:
# Phase 0: コンテクスト抽出（対話形式）
# このセルは必要に応じて実行してください

# user_context = pipeline.run_phase0_context_extraction()
# print("✓ Phase 0 完了")

## Cell 5: Phase 1 - 100倍拡張

コンテクストを拡張し、100個の願望・能力・役割リストを生成します。

**注意**: この処理には5〜25分程度かかります（ハードウェアに依存）。

In [ ]:
# Phase 1: 100倍拡張
print("Phase 1: 100倍拡張を開始します...")
print("この処理には数分かかる場合があります。\n")

phase1_results = pipeline.run_phase1_expansion(user_context)

# 結果の表示
print("\n" + "="*60)
print("Phase 1 完了")
print("="*60)

if "desire_list" in phase1_results:
    print("\n✓ 願望リスト生成完了")
    
if "ability_list" in phase1_results:
    print("✓ 能力リスト生成完了")
    
if "role_list" in phase1_results:
    print("✓ 役割リスト生成完了")
    
if "plottype_list" in phase1_results:
    print("✓ プロットタイプリスト生成完了")
    
if "plottype" in phase1_results:
    print("✓ プロットタイプ選択完了")

print("\n中間データは ./output/intermediate/ に保存されました")

## Cell 6: 生成されたリストの確認

Phase 1で生成されたデータを確認します。

In [ ]:
# 生成されたデータの確認
from src.utils import load_yaml

# 願望リストの最初の10個を表示
desire_data = load_yaml("./output/intermediate/01_desire_list.yaml")
if desire_data and "desires" in desire_data:
    print("【願望リスト（最初の10個）】")
    for i, desire in enumerate(desire_data["desires"][:10], 1):
        print(f"{i}. {desire}")
    print(f"... 全{len(desire_data['desires'])}個\n")

# 選択されたプロットタイプを表示
plottype_data = load_yaml("./output/intermediate/05_plottype.yaml")
if plottype_data and "selected_plottype" in plottype_data:
    plottype = plottype_data["selected_plottype"]
    print("【選択されたプロットタイプ】")
    print(f"タイプ: {plottype.get('plot_type', 'N/A')}")
    print(f"構造: {plottype.get('core_structure', 'N/A')[:100]}...")

## Cell 7: 完全パイプラインの実行（オプション）

Phase 2以降を含む完全なパイプラインを実行する場合は、このセルを使用します。

**警告**: 完全なパイプラインの実行には1.5〜8時間かかります。

In [ ]:
# 完全パイプラインの実行（コメントアウトを解除して実行）
# 
# print("完全パイプラインを実行します...")
# print("この処理には数時間かかる場合があります。\n")
# 
# all_results = pipeline.run_full_pipeline(user_context)
# 
# print("\n" + "="*60)
# print("完全パイプライン実行完了")
# print("="*60)
# print(f"\n生成された小説は ./output/novels/ に保存されました")
# print(f"設定資料集は ./output/references/ に保存されました")

## Cell 8: チェックポイントからの再開

中断した場合や、特定のフェーズから再開したい場合に使用します。

In [ ]:
# チェックポイントのリスト表示
checkpoints = pipeline.checkpoint_manager.list_checkpoints()

print("利用可能なチェックポイント:")
print("="*60)
for i, cp in enumerate(checkpoints[:10], 1):  # 最新10個を表示
    print(f"{i}. {Path(cp).name}")

if not checkpoints:
    print("（チェックポイントがありません）")

# チェックポイントから再開する例
# pipeline.resume_from_checkpoint("phase1_expansion")

## Cell 9: 状態の確認とエクスポート

現在のパイプライン状態を確認します。

In [ ]:
# 現在の状態を表示
state_summary = pipeline.checkpoint_manager.export_state_summary()
print(state_summary)

# 出力ディレクトリの内容を確認
print("\n" + "="*60)
print("出力ファイル一覧")
print("="*60)

output_dir = Path("./output")
if output_dir.exists():
    for subdir in ["intermediate", "checkpoints", "novels", "references"]:
        subdir_path = output_dir / subdir
        if subdir_path.exists():
            files = list(subdir_path.glob("*"))
            print(f"\n{subdir}/: {len(files)}ファイル")
            for f in sorted(files)[:5]:  # 最初の5個を表示
                print(f"  - {f.name}")
            if len(files) > 5:
                print(f"  ... 他 {len(files) - 5}ファイル")
else:
    print("出力ディレクトリがまだ作成されていません")

## Cell 10: クリーンアップ（オプション）

チェックポイントや一時ファイルをクリーンアップします。

**警告**: このセルを実行すると、保存されたデータが削除されます。

In [ ]:
# クリーンアップ（コメントアウトを解除して実行）
#
# import shutil
#
# # チェックポイントの削除
# response = input("すべてのチェックポイントを削除しますか？ (yes/no): ")
# if response.lower() == "yes":
#     checkpoint_dir = Path("./output/checkpoints")
#     if checkpoint_dir.exists():
#         shutil.rmtree(checkpoint_dir)
#         checkpoint_dir.mkdir(parents=True)
#         print("✓ チェックポイントを削除しました")
#
# # 中間データの削除
# response = input("すべての中間データを削除しますか？ (yes/no): ")
# if response.lower() == "yes":
#     intermediate_dir = Path("./output/intermediate")
#     if intermediate_dir.exists():
#         shutil.rmtree(intermediate_dir)
#         intermediate_dir.mkdir(parents=True)
#         print("✓ 中間データを削除しました")

---

## 完了

お疲れ様でした！生成された小説と設定資料集は `./output/` ディレクトリに保存されています。

### 次のステップ

1. `./output/novels/` - 生成された小説を確認
2. `./output/references/` - 詳細な設定資料集を確認
3. 必要に応じて、パラメータを調整して再実行

### サポート

- [GitHub Issues](https://github.com/masa-jp-art/100-times-ai-world-building/issues)
- [README_LOCAL.md](README_LOCAL.md)

---

**Version**: v2.0-local  
**Author**: masa-jp-art  
**Last Updated**: 2026-02-14